In [1]:
from restrictions import *

In [ ]:
# Define or reset the food categories
FoodCategory.reset()
FoodCategory.define("ANIMAL_PRODUCTS")
FoodCategory.define("MEAT", {"ANIMAL_PRODUCTS"})
FoodCategory.define("DAIRY", {"ANIMAL_PRODUCTS"})
FoodCategory.define("EGGS", {"ANIMAL_PRODUCTS"})
FoodCategory.define("FISH", {"ANIMAL_PRODUCTS"})
FoodCategory.define("BEEF", {"MEAT"})
FoodCategory.define("CHEESE", {"DAIRY"})
FoodCategory.define("CHICKEN", {"MEAT"})

# Define a dietary restriction
vegan = DietaryRestriction({"ANIMAL_PRODUCTS"})

# Create a meal
cheeseburger = Meal("Cheeseburger", [
    FoodCategory.get("BEEF"),
    FoodCategory.get("CHEESE")
])

# Compatibility check
print(cheeseburger)                                      
print("Categories:", cheeseburger.categories())          
print("Vegan compatible?", cheeseburger.is_compatible_with(vegan))  


Meal(Cheeseburger: [BEEF, CHEESE])
Categories: {'MEAT', 'CHEESE', 'DAIRY', 'ANIMAL_PRODUCTS', 'BEEF'}
Vegan compatible? False


In [ ]:
# Define dietary restrictions
vegan = DietaryRestriction({"ANIMAL_PRODUCTS"})

In [3]:
beef = FoodCategory.get("BEEF")
print(beef.is_a("MEAT"))            # True
print(beef.is_a("ANIMAL_PRODUCTS")) # True
print(beef.is_a("NUTS"))            # False

animal_products = FoodCategory.get("ANIMAL_PRODUCTS")
print(animal_products.all())            # set()


True
True
False
[FoodCategory(ANIMAL_PRODUCTS), FoodCategory(MEAT), FoodCategory(DAIRY), FoodCategory(EGGS), FoodCategory(FISH), FoodCategory(BEEF), FoodCategory(CHEESE), FoodCategory(CHICKEN)]


In [4]:
vegan = DietaryRestriction({"ANIMAL_PRODUCTS"})
vegetarian = DietaryRestriction({"MEAT", "FISH", "SHELLFISH"})

meal = [FoodCategory.get("CHEESE"), FoodCategory.get("CHICKEN")]

print("Vegan compatible:", vegan.is_compatible_with(meal))        # False
print("Vegetarian compatible:", vegetarian.is_compatible_with(meal))  # False


Vegan compatible: False
Vegetarian compatible: False


In [5]:
cheese = FoodCategory.get("CHEESE")
print(cheese.ancestors())  # Expected: {'DAIRY', 'ANIMAL_PRODUCTS'}


{'DAIRY', 'ANIMAL_PRODUCTS'}
